In [1]:
#Importing Libraries
import re
import numpy as np
import pandas as pd
import datetime

# For Private Residential Properties

Using data-1.json

In [2]:
#Loading in Dataset and making Dataframe

pvtResidential = pd.read_csv('data-1.csv')
pvtResidential.head()

,Status,Result.street,Result.x,Result.project,Result.y,Result.marketSegment,Result.transaction.area,Result.transaction.floorRange,Result.transaction.noOfUnits,Result.transaction.contractDate,Result.transaction.typeOfSale,Result.transaction.price,Result.transaction.propertyType,Result.transaction.district,Result.transaction.typeOfArea,Result.transaction.tenure,Result.transaction.nettPrice
0,Success,NEO PEE TECK LANE,20914.198897,LANDED HOUSING DEVELOPMENT,30589.107079,RCR,257.0,-,1,522,3,4600000,Terrace,5,Land,Freehold,NaN
1,Success,NEO PEE TECK LANE,20914.198897,LANDED HOUSING DEVELOPMENT,30589.107079,RCR,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN
2,Success,NEO PEE TECK LANE,20914.198897,LANDED HOUSING DEVELOPMENT,30589.107079,RCR,186.0,-,1,321,3,1880000,Strata Terrace,5,Strata,Freehold,NaN
3,Success,ZEHNDER ROAD,22734.925300,LANDED HOUSING DEVELOPMENT,29502.420710,RCR,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN
4,Success,ZEHNDER ROAD,22734.925300,LANDED HOUSING DEVELOPMENT,29502.420710,RCR,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN


In [3]:
#Cleaning data

pvtResidential.fillna(0, inplace = True)

#Dropping irrelevant columns

newpvtResidential = pvtResidential.drop(['Status','Result.street','Result.transaction.typeOfArea','Result.transaction.noOfUnits','Result.transaction.area','Result.transaction.nettPrice','Result.transaction.floorRange','Result.transaction.contractDate','Result.transaction.typeOfSale', 'Result.x', 'Result.project', 'Result.y', 'Result.marketSegment'], axis = 1)

print(newpvtResidential.head())

   Result.transaction.price Result.transaction.propertyType  \
0                   4600000                         Terrace   
1                   2630000                         Terrace   
2                   1880000                  Strata Terrace   
3                   5500000                   Semi-detached   
4                   5000000                   Semi-detached   

   Result.transaction.district Result.transaction.tenure  
0                            5                  Freehold  
1                            5                  Freehold  
2                            5                  Freehold  
3                            5                  Freehold  
4                            5                  Freehold  


In [4]:
newpvtResidential.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23600 entries, 0 to 23599
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Result.transaction.price         23600 non-null  int64 
 1   Result.transaction.propertyType  23600 non-null  object
 2   Result.transaction.district      23600 non-null  int64 
 3   Result.transaction.tenure        23600 non-null  object
dtypes: int64(2), object(2)
memory usage: 737.6+ KB


In [5]:
#Renaming the columns

newpvtResidential.rename(columns = {'Result.transaction.price':'propertyPrice'}, inplace = True)
newpvtResidential.rename(columns = {'Result.transaction.propertyType':'propertyType'}, inplace = True)
newpvtResidential.rename(columns = {'Result.transaction.tenure':'propertyTenure'}, inplace = True)
newpvtResidential.rename(columns = {'Result.transaction.district':'districtNo'}, inplace = True)
newpvtResidential.head()

,propertyPrice,propertyType,districtNo,propertyTenure
0,4600000,Terrace,5,Freehold
1,2630000,Terrace,5,Freehold
2,1880000,Strata Terrace,5,Freehold
3,5500000,Semi-detached,5,Freehold
4,5000000,Semi-detached,5,Freehold


In [6]:
#Inserting the new column propertyPrivatePublic

newpvtResidential.insert(4, 'propertyPrivatePublic', 'Private')

newpvtResidential.insert(5, 'statusBuyRent', 'Buy')

newpvtResidential.head()


,propertyPrice,propertyType,districtNo,propertyTenure,propertyPrivatePublic,statusBuyRent
0,4600000,Terrace,5,Freehold,Private,Buy
1,2630000,Terrace,5,Freehold,Private,Buy
2,1880000,Strata Terrace,5,Freehold,Private,Buy
3,5500000,Semi-detached,5,Freehold,Private,Buy
4,5000000,Semi-detached,5,Freehold,Private,Buy


In [7]:
#Converting Freehold and 99 yrs lease to numeric values.
def convert_to_years(lease_term):
    if isinstance(lease_term, str):
        if 'lease commencing from' in lease_term:
            start_year_str = lease_term.split()[-1]
            if start_year_str.isdigit():
                start_year = int(start_year_str)
                remaining_years = 99 - (datetime.datetime.now().year - start_year)
                return str(remaining_years)
        elif lease_term == 'Freehold':
            return '999'
    return lease_term


newpvtResidential['propertyTenure'] = newpvtResidential['propertyTenure'].apply(convert_to_years)
newpvtResidential['propertyTenure'] = pd.to_numeric(newpvtResidential['propertyTenure'])
newpvtResidential['propertyTenure'] = newpvtResidential['propertyTenure'].apply(float)
newpvtResidential['propertyTenure'] = newpvtResidential['propertyTenure'].fillna(0)


newpvtResidential

,propertyPrice,propertyType,districtNo,propertyTenure,propertyPrivatePublic,statusBuyRent
0,4600000,Terrace,5,999.0,Private,Buy
1,2630000,Terrace,5,999.0,Private,Buy
2,1880000,Strata Terrace,5,999.0,Private,Buy
3,5500000,Semi-detached,5,999.0,Private,Buy
4,5000000,Semi-detached,5,999.0,Private,Buy
...,...,...,...,...,...,...
23595,1988000,Condominium,3,89.0,Private,Buy
23596,1500000,Condominium,3,89.0,Private,Buy
23597,1086000,Condominium,3,89.0,Private,Buy
23598,1150000,Condominium,3,89.0,Private,Buy


# For HDB

In [8]:
#Loading in Dataset

hdbDetails = pd.read_csv('data-5-govsg.csv')
hdbDetails.head()

,/_id,/block,/flat_model,/flat_type,/floor_area_sqm,/lease_commence_date,/month,/remaining_lease,/resale_price,/storey_range,/street_name,/town
0,1,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,2,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,3,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,4,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,5,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [9]:
#Cleaning data

hdbDetails.fillna(0, inplace = True)

#Renaming Column Headings

hdbDetails.rename(columns = {'/_id':'ID'}, inplace = True)
hdbDetails.rename(columns = {'/block':'Block No.'}, inplace = True)
hdbDetails.rename(columns = {'/flat_model':'Flat Type'}, inplace = True)
hdbDetails.rename(columns = {'/floor_area_sqm':'Square Meters'}, inplace = True)
hdbDetails.rename(columns = {'/lease_commence_date':'Lease Start Date'}, inplace = True)
hdbDetails.rename(columns = {'/month':'Month'}, inplace = True)
hdbDetails.rename(columns = {'/remaining_lease':'Remaining Years of Lease'}, inplace = True)
hdbDetails.rename(columns = {'/resale_price':'Price'}, inplace = True)
hdbDetails.rename(columns = {'/storey_range':'Storey Range'}, inplace = True)
hdbDetails.rename(columns = {'/street_name':'Street Name'}, inplace = True)
hdbDetails.rename(columns = {'/town':'Area'}, inplace = True)

hdbDetails.head()

,ID,Block No.,Flat Type,/flat_type,Square Meters,Lease Start Date,Month,Remaining Years of Lease,Price,Storey Range,Street Name,Area
0,1,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,2,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,3,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,4,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,5,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [10]:
#Combining Columns Flat Type and /Flat_type to new column PropertyType

hdbDetails['propertyType'] = hdbDetails['Flat Type'] + ' ' + hdbDetails['/flat_type']

hdbDetails

,ID,Block No.,Flat Type,/flat_type,Square Meters,Lease Start Date,Month,Remaining Years of Lease,Price,Storey Range,Street Name,Area,propertyType
0,1,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO,Improved 2 ROOM
1,2,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO,New Generation 3 ROOM
2,3,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO,New Generation 3 ROOM
3,4,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO,New Generation 3 ROOM
4,5,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO,New Generation 3 ROOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99991,183B,Model A,4 ROOM,92.0,2015,2021-07,92 years 06 months,500000.0,16 TO 18,BOON LAY AVE,JURONG WEST,Model A 4 ROOM
99996,99992,656C,Improved,5 ROOM,112.0,2001,2021-07,79 years 01 month,500000.0,01 TO 03,JURONG WEST ST 61,JURONG WEST,Improved 5 ROOM
99997,99993,183B,Model A,4 ROOM,92.0,2015,2021-07,92 years 07 months,500000.0,04 TO 06,BOON LAY AVE,JURONG WEST,Model A 4 ROOM
99998,99994,678C,Premium Apartment,5 ROOM,110.0,2008,2021-07,85 years 11 months,505000.0,13 TO 15,JURONG WEST ST 64,JURONG WEST,Premium Apartment 5 ROOM


In [11]:
#Dropping irrelevant columns

newhdbDetails = hdbDetails.drop(['ID','Block No.','Flat Type','/flat_type','Square Meters','Lease Start Date','Month', 'Street Name'], axis = 1)

print(newhdbDetails.head())

  Remaining Years of Lease     Price Storey Range        Area  \
0       61 years 04 months  232000.0     10 TO 12  ANG MO KIO   
1       60 years 07 months  250000.0     01 TO 03  ANG MO KIO   
2       62 years 05 months  262000.0     01 TO 03  ANG MO KIO   
3        62 years 01 month  265000.0     04 TO 06  ANG MO KIO   
4       62 years 05 months  265000.0     01 TO 03  ANG MO KIO   

            propertyType  
0        Improved 2 ROOM  
1  New Generation 3 ROOM  
2  New Generation 3 ROOM  
3  New Generation 3 ROOM  
4  New Generation 3 ROOM  


In [12]:
#Renaming columns

newhdbDetails.rename(columns = {'Remaining Years of Lease':'propertyTenure'}, inplace = True)
newhdbDetails.rename(columns = {'Price':'propertyPrice'}, inplace = True)
newhdbDetails.rename(columns = {'Storey Range':'propertyLevels'}, inplace = True)
newhdbDetails.rename(columns = {'Area':'areaName'}, inplace = True)

newhdbDetails

,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType
0,61 years 04 months,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM
1,60 years 07 months,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
2,62 years 05 months,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
3,62 years 01 month,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM
4,62 years 05 months,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
...,...,...,...,...,...
99995,92 years 06 months,500000.0,16 TO 18,JURONG WEST,Model A 4 ROOM
99996,79 years 01 month,500000.0,01 TO 03,JURONG WEST,Improved 5 ROOM
99997,92 years 07 months,500000.0,04 TO 06,JURONG WEST,Model A 4 ROOM
99998,85 years 11 months,505000.0,13 TO 15,JURONG WEST,Premium Apartment 5 ROOM


In [13]:
newhdbDetails.dtypes

propertyTenure     object
propertyPrice     float64
propertyLevels     object
areaName           object
propertyType       object
dtype: object

In [14]:
#Cleaning Data
#Ensuring that for values like "61 years", change it to "61 years and 0 months"
#Useful for the next step

def add_months(text):
    if "year" in text and "month" not in text:
        return text + " and 0 months"
    else:
        return text
    
    
newhdbDetails['propertyTenure'] = newhdbDetails['propertyTenure'].apply(add_months)

newhdbDetails

,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType
0,61 years 04 months,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM
1,60 years 07 months,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
2,62 years 05 months,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
3,62 years 01 month,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM
4,62 years 05 months,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
...,...,...,...,...,...
99995,92 years 06 months,500000.0,16 TO 18,JURONG WEST,Model A 4 ROOM
99996,79 years 01 month,500000.0,01 TO 03,JURONG WEST,Improved 5 ROOM
99997,92 years 07 months,500000.0,04 TO 06,JURONG WEST,Model A 4 ROOM
99998,85 years 11 months,505000.0,13 TO 15,JURONG WEST,Premium Apartment 5 ROOM


In [15]:
#Converting the propertyTenure column to decimal format in terms of years

def convert_to_decimal(text):
    # Define regular expression to match numerical values
    pattern = r'\d+'
    # Extract numerical values for years and months using regular expression
    matches = re.findall(pattern, text)
    years = int(matches[0])
    months = int(matches[1])
    decimal_value = years + months/12
    return round(decimal_value,2)

newhdbDetails['propertyTenure'] = newhdbDetails['propertyTenure'].apply(convert_to_decimal)

newhdbDetails

,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType
0,61.33,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM
1,60.58,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
2,62.42,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
3,62.08,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM
4,62.42,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM
...,...,...,...,...,...
99995,92.50,500000.0,16 TO 18,JURONG WEST,Model A 4 ROOM
99996,79.08,500000.0,01 TO 03,JURONG WEST,Improved 5 ROOM
99997,92.58,500000.0,04 TO 06,JURONG WEST,Model A 4 ROOM
99998,85.92,505000.0,13 TO 15,JURONG WEST,Premium Apartment 5 ROOM


In [16]:
# create a dictionary of town names and their corresponding district numbers
district_mapping = {
    'RAFFLES PLACE': 1,
    'MARINA': 1,
    'CECIL': 1,
    'TANJONG PAGAR': 2,
    'CHINATOWN': 2,
    'ALEXANDRA': 3,
    'QUEENSTOWN': 3,
    'TIONG BAHRU': 3,
    'TELOK BLANGAH': 4,
    'HARBOURFRONT': 4,
    'BUONA VISTA': 5,
    'PASIR PANJANG ': 5,
    'CLEMENTI': 5,
    'HIGH STREET': 6,
    'BEACH ROAD': 6,
    'MIDDLE ROAD': 7,
    'GOLDEN MILE': 7,
    'LITTLE INDIA': 8,
    'ORCHARD ROAD': 9,
    'RIVER VALLEY': 9,
    'TANGLIN': 10,
    'HOLLAND': 10,
    'WATTEN ESTATE': 11,
    'NOVEN': 11,
    'THOMSON': 11,
    'BALESTIER': 12,
    'TOA PAYOH': 12,
    'SERANGOON': 12,
    'MACPHERSON': 13,
    'BRADDELL': 13,
    'GEYLANG': 14,
    'PAYA LEBAR': 14,
    'EUNOS': 14,
    'KEMBANGAN': 14,
    'BEDOK': 16,
    'UPPER EAST COAST': 16,
    'EASTWOOD': 16,
    'KEW DRIVE': 16,
    'LOYANG': 17,
    'CHANGI': 17,
    'TAMPINES': 18,
    'PASIR RIS': 18,
    'SERANGOON GARDENS': 19,
    'HOUGANG': 19,
    'PUNGGOL': 19,
    'BISHAN': 20,
    'ANG MO KIO': 20,
    'UPPER BUKIT TIMAH': 21,
    'CLEMENTI PARK': 21,
    'ULU PANDAN': 21,
    'JURONG': 22,
    'HILLVIEW': 23,
    'DAIRY FARM': 23,
    'BUKIT PANJANG': 23,
    'CHOA CHU KANG': 23,
    'LIM CHU KANG': 24,
    'TENGAH': 24,
    'KRANJI': 25,
    'WOODGROVE': 25,
    'WOODLANDS': 25,
    'UPPER THOMSON': 26,
    'SPRINGLEAF': 26,
    'YISHUN': 27,
    'SEMBAWANG': 27,
    'SELETAR': 28,
    'YIO CHU KANG': 28,
    'SENGKANG': 28,
    'JURONG WEST': 22,
    'JURONG EAST': 22
}

# create a new column called 'district_number' and map the town names to their corresponding district numbers
newhdbDetails['districtNo'] = newhdbDetails['areaName'].map(district_mapping)

newhdbDetails

,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType,districtNo
0,61.33,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM,20.0
1,60.58,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20.0
2,62.42,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20.0
3,62.08,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM,20.0
4,62.42,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20.0
...,...,...,...,...,...,...
99995,92.50,500000.0,16 TO 18,JURONG WEST,Model A 4 ROOM,22.0
99996,79.08,500000.0,01 TO 03,JURONG WEST,Improved 5 ROOM,22.0
99997,92.58,500000.0,04 TO 06,JURONG WEST,Model A 4 ROOM,22.0
99998,85.92,505000.0,13 TO 15,JURONG WEST,Premium Apartment 5 ROOM,22.0


In [17]:
#converting districtNo from float to int64

#Replacingnon-finite values with a default value 0

newhdbDetails['districtNo'] = newhdbDetails['districtNo'].fillna(0)

newhdbDetails['districtNo'] = newhdbDetails['districtNo'].astype(int)

newhdbDetails.head()


,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType,districtNo
0,61.33,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM,20
1,60.58,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20
2,62.42,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20
3,62.08,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM,20
4,62.42,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,20


In [18]:
#Inserting new column PropertyPrivatePublic

newhdbDetails.insert(5, 'propertyPrivatePublic', 'Public')
newhdbDetails



,propertyTenure,propertyPrice,propertyLevels,areaName,propertyType,propertyPrivatePublic,districtNo
0,61.33,232000.0,10 TO 12,ANG MO KIO,Improved 2 ROOM,Public,20
1,60.58,250000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,Public,20
2,62.42,262000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,Public,20
3,62.08,265000.0,04 TO 06,ANG MO KIO,New Generation 3 ROOM,Public,20
4,62.42,265000.0,01 TO 03,ANG MO KIO,New Generation 3 ROOM,Public,20
...,...,...,...,...,...,...,...
99995,92.50,500000.0,16 TO 18,JURONG WEST,Model A 4 ROOM,Public,22
99996,79.08,500000.0,01 TO 03,JURONG WEST,Improved 5 ROOM,Public,22
99997,92.58,500000.0,04 TO 06,JURONG WEST,Model A 4 ROOM,Public,22
99998,85.92,505000.0,13 TO 15,JURONG WEST,Premium Apartment 5 ROOM,Public,22


In [19]:
#Inserting new column statusBuyRent
NewhdbDetails = newhdbDetails.drop(['areaName'], axis = 1)
NewhdbDetails.insert(6, 'statusBuyRent', 'Buy')
NewhdbDetails

,propertyTenure,propertyPrice,propertyLevels,propertyType,propertyPrivatePublic,districtNo,statusBuyRent
0,61.33,232000.0,10 TO 12,Improved 2 ROOM,Public,20,Buy
1,60.58,250000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
2,62.42,262000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
3,62.08,265000.0,04 TO 06,New Generation 3 ROOM,Public,20,Buy
4,62.42,265000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
...,...,...,...,...,...,...,...
99995,92.50,500000.0,16 TO 18,Model A 4 ROOM,Public,22,Buy
99996,79.08,500000.0,01 TO 03,Improved 5 ROOM,Public,22,Buy
99997,92.58,500000.0,04 TO 06,Model A 4 ROOM,Public,22,Buy
99998,85.92,505000.0,13 TO 15,Premium Apartment 5 ROOM,Public,22,Buy


# For Private Non-Landed Residential Property

In [20]:
#Loading in dataset

nonLanded = pd.read_csv('data-2.csv')
nonLanded

,Status,Result.street,Result.x,Result.project,Result.y,Result.rentalMedian.refPeriod,Result.rentalMedian.psf75,Result.rentalMedian.median,Result.rentalMedian.psf25,Result.rentalMedian.district
0,Success,HILLVIEW AVENUE,19735.41158,MERAWOODS,37530.81327,2020Q3,2.01,1.93,1.84,23
1,Success,HILLVIEW AVENUE,19735.41158,MERAWOODS,37530.81327,2020Q1,1.94,1.91,1.82,23
2,Success,HILLVIEW AVENUE,19735.41158,MERAWOODS,37530.81327,2021Q4,2.40,2.17,1.98,23
3,Success,HILLVIEW AVENUE,19735.41158,MERAWOODS,37530.81327,2021Q2,2.28,2.06,1.86,23
4,Success,HILLVIEW AVENUE,19735.41158,MERAWOODS,37530.81327,2022Q1,2.43,2.07,1.94,23
...,...,...,...,...,...,...,...,...,...,...
7485,Success,SIN MING ROAD,28255.63458,THOMSON V TWO,37271.68720,2021Q4,4.74,4.43,4.07,20
7486,Success,SIN MING ROAD,28255.63458,THOMSON V TWO,37271.68720,2021Q1,4.36,3.91,3.86,20
7487,Success,SIN MING ROAD,28255.63458,THOMSON V TWO,37271.68720,2022Q3,6.00,4.97,4.79,20
7488,Success,PERUMAL ROAD,30477.87296,UPTOWN @ FARRER,32796.58779,2022Q2,6.83,6.54,5.89,8


In [21]:
#Cleaning data

nonLanded.fillna(0, inplace = True)

#Dropping irrelevant columns

newnonLanded = nonLanded.drop(['Status','Result.street','Result.x','Result.y','Result.rentalMedian.psf25','Result.rentalMedian.psf75'], axis = 1)

print(newnonLanded.head())

  Result.project Result.rentalMedian.refPeriod  Result.rentalMedian.median  \
0      MERAWOODS                        2020Q3                        1.93   
1      MERAWOODS                        2020Q1                        1.91   
2      MERAWOODS                        2021Q4                        2.17   
3      MERAWOODS                        2021Q2                        2.06   
4      MERAWOODS                        2022Q1                        2.07   

   Result.rentalMedian.district  
0                            23  
1                            23  
2                            23  
3                            23  
4                            23  


In [22]:
#Renaming Columns

newnonLanded.rename(columns = {'Result.project':'propertyProjectName'}, inplace = True)
newnonLanded.rename(columns = {'Result.rentalMedian.median':'rentalPriceSqft'}, inplace = True)
newnonLanded.rename(columns = {'Result.rentalMedian.refPeriod':'rentalRefPeriod'}, inplace = True)
newnonLanded.rename(columns = {'Result.rentalMedian.district':'districtNo'}, inplace = True)

newnonLanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7490 entries, 0 to 7489
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   propertyProjectName  7490 non-null   object 
 1   rentalRefPeriod      7490 non-null   object 
 2   rentalPriceSqft      7490 non-null   float64
 3   districtNo           7490 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 234.2+ KB


In [23]:
#Changing the values in rentalPriceSqft to rentalPriceSqft*1000 
newnonLanded['rentalPriceSqft'] = newnonLanded['rentalPriceSqft'].apply(lambda x: x*1000)

newnonLanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7490 entries, 0 to 7489
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   propertyProjectName  7490 non-null   object 
 1   rentalRefPeriod      7490 non-null   object 
 2   rentalPriceSqft      7490 non-null   float64
 3   districtNo           7490 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 234.2+ KB


In [24]:
#Inserting new columns into the dataframe

newnonLanded.insert(4, 'statusBuyRent', 'Rent')
newnonLanded.insert(5, 'propertyPrivatePublic', 'Private')

newnonLanded

,propertyProjectName,rentalRefPeriod,rentalPriceSqft,districtNo,statusBuyRent,propertyPrivatePublic
0,MERAWOODS,2020Q3,1930.0,23,Rent,Private
1,MERAWOODS,2020Q1,1910.0,23,Rent,Private
2,MERAWOODS,2021Q4,2170.0,23,Rent,Private
3,MERAWOODS,2021Q2,2060.0,23,Rent,Private
4,MERAWOODS,2022Q1,2070.0,23,Rent,Private
...,...,...,...,...,...,...
7485,THOMSON V TWO,2021Q4,4430.0,20,Rent,Private
7486,THOMSON V TWO,2021Q1,3910.0,20,Rent,Private
7487,THOMSON V TWO,2022Q3,4970.0,20,Rent,Private
7488,UPTOWN @ FARRER,2022Q2,6540.0,8,Rent,Private


# For Private Residential Project in the Pipeline

In [25]:
#Loading in dataset

projPipeline = pd.read_csv('data-4.csv')
projPipeline.head()

,Status,Result.noOfApartment,Result.expectedTOPYear,Result.noOfSemiDetached,Result.street,Result.district,Result.project,Result.noOfDetachedHouse,Result.noOfTerrace,Result.noOfCondo,Result.developerName,Result.totalUnits
0,Success,0,na,0,Dairy Farm Walk,23,The Botany At Dairy Farm,0,0,386,Sim Lian JV (Dairy Farm) Pte Ltd,386
1,Success,154,na,0,Orchard Boulevard,10,Boulevard 88,0,0,0,Granmil Holdings Pte Ltd,154
2,Success,1451,na,0,Hougang Avenue 7,19,Riverfront Residences,0,21,0,Rio Casa Venture Pte Ltd,1472
3,Success,1410,na,0,Hougang Avenue 2,19,The Florence Residences,0,0,0,Florence Development Pte Ltd,1410
4,Success,1840,na,0,Normanton Park,5,Normanton Park,0,22,0,Kingsford Huray Development Pte Ltd,1862


In [26]:
#Cleaning data

projPipeline.fillna(0, inplace = True)

#Dropping irrelevant columns

newprojPipeline = projPipeline.drop(['Status','Result.expectedTOPYear','Result.noOfApartment','Result.noOfSemiDetached','Result.street','Result.noOfDetachedHouse','Result.noOfTerrace','Result.noOfCondo'], axis = 1)

#Renaming columns
newprojPipeline.rename(columns = {'Result.district':'districtNo'}, inplace = True)
newprojPipeline.rename(columns = {'Result.project':'propertyProjectName'}, inplace = True)
newprojPipeline.rename(columns = {'Result.developerName':'propertyDeveloperName'}, inplace = True)
newprojPipeline.rename(columns = {'Result.totalUnits':'propertyUnitsAvail'}, inplace = True)


newprojPipeline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   districtNo             82 non-null     int64 
 1   propertyProjectName    82 non-null     object
 2   propertyDeveloperName  82 non-null     object
 3   propertyUnitsAvail     82 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.7+ KB


In [27]:
#Inserting new columns
newprojPipeline.insert(4, 'statusBuyRent', 'Buy')
newprojPipeline.insert(5, 'propertyPrivatePublic', 'Private')

newprojPipeline

,districtNo,propertyProjectName,propertyDeveloperName,propertyUnitsAvail,statusBuyRent,propertyPrivatePublic
0,23,The Botany At Dairy Farm,Sim Lian JV (Dairy Farm) Pte Ltd,386,Buy,Private
1,10,Boulevard 88,Granmil Holdings Pte Ltd,154,Buy,Private
2,19,Riverfront Residences,Rio Casa Venture Pte Ltd,1472,Buy,Private
3,19,The Florence Residences,Florence Development Pte Ltd,1410,Buy,Private
4,5,Normanton Park,Kingsford Huray Development Pte Ltd,1862,Buy,Private
...,...,...,...,...,...,...
77,1,Skywaters Residences,Perennial Shenton Property Pte Ltd,215,Buy,Private
78,24,Copen Grand,Taurus Properties SG Pte Ltd,639,Buy,Private
79,18,Tenet,TQS Development Pte Ltd,618,Buy,Private
80,26,Lentor Modern,Lentor Central Pte Ltd/Lentor Modern Pte Ltd,605,Buy,Private


# For Private Residentials Property (Sold by Developers)


In [28]:
#Loading in dataset from data-3
unitSold = pd.read_csv("data-3.csv")
unitSold.head()

,Status,Result.street,Result.district,Result.propertyType,Result.x,Result.project,Result.y,Result.developer,Result.marketSegment,Result.developerSales.refPeriod,Result.developerSales.medianPrice,Result.developerSales.highestPrice,Result.developerSales.lowestPrice,Result.developerSales.launchedToDate,Result.developerSales.soldInMonth,Result.developerSales.launchedInMonth,Result.developerSales.soldToDate,Result.developerSales.unitsAvail
0,Success,LORONG H TELOK KURAU,15,Non-Landed,36702.621554,RYMDEN 77,33212.588605,QHS Development Pte Ltd,OCR,123,0,0,0,31,0,0,30,31
1,Success,SILAT AVENUE,3,Non-Landed,27539.617540,AVENUE SOUTH RESIDENCE,28778.744350,United Venture Development (Silat) Pte Ltd,RCR,123,0,0,0,1074,0,0,1074,1074
2,Success,HOUGANG AVENUE 2,19,Non-Landed,33973.310030,THE FLORENCE RESIDENCES,38799.795650,Florence Development Pte Ltd,OCR,123,0,0,0,1410,0,0,1410,1410
3,Success,HOLLAND ROAD,10,Non-Landed,24512.843400,HYLL ON HOLLAND,32524.393720,FEC Skypark Pte Ltd,CCR,123,2788,2926,2603,319,8,0,251,319
4,Success,JIAK KIM STREET,3,Non-Landed,28224.283540,RIVIERE,30462.176050,Frasers Property Quayside Pte Ltd,RCR,123,3087,3274,2760,455,13,35,421,455


In [29]:
#Loading in dataset from data-3-1

unitSold1 = pd.read_csv("data-3-1.csv")
unitSold1.head()

,Status,Result.street,Result.district,Result.propertyType,Result.x,Result.project,Result.y,Result.developer,Result.marketSegment,Result.developerSales.refPeriod,Result.developerSales.medianPrice,Result.developerSales.highestPrice,Result.developerSales.lowestPrice,Result.developerSales.launchedToDate,Result.developerSales.soldInMonth,Result.developerSales.launchedInMonth,Result.developerSales.soldToDate,Result.developerSales.unitsAvail
0,Success,LORONG H TELOK KURAU,15,Non-Landed,36702.621554,RYMDEN 77,33212.588605,QHS Development Pte Ltd,OCR,1222,0,0,0,31,0,0,30,31
1,Success,SILAT AVENUE,3,Non-Landed,27539.617540,AVENUE SOUTH RESIDENCE,28778.744350,United Venture Development (Silat) Pte Ltd,RCR,1222,0,0,0,1074,0,0,1074,1074
2,Success,HOUGANG AVENUE 2,19,Non-Landed,33973.310030,THE FLORENCE RESIDENCES,38799.795650,Florence Development Pte Ltd,OCR,1222,0,0,0,1410,0,0,1410,1410
3,Success,HOLLAND ROAD,10,Non-Landed,24512.843400,HYLL ON HOLLAND,32524.393720,FEC Skypark Pte Ltd,CCR,1222,2675,2971,2503,319,7,0,243,319
4,Success,JIAK KIM STREET,3,Non-Landed,28224.283540,RIVIERE,30462.176050,Frasers Property Quayside Pte Ltd,RCR,1222,2978,3386,2743,420,14,20,410,455


In [30]:
#Loading in dataset from data-3-2

unitSold2 = pd.read_csv("data-3-2.csv")
unitSold2.head()

,Status,Result.street,Result.district,Result.propertyType,Result.x,Result.project,Result.y,Result.developer,Result.marketSegment,Result.developerSales.refPeriod,Result.developerSales.medianPrice,Result.developerSales.highestPrice,Result.developerSales.lowestPrice,Result.developerSales.launchedToDate,Result.developerSales.soldInMonth,Result.developerSales.launchedInMonth,Result.developerSales.soldToDate,Result.developerSales.unitsAvail
0,Success,LORONG H TELOK KURAU,15,Non-Landed,36702.621554,RYMDEN 77,33212.588605,QHS Development Pte Ltd,OCR,1122,0,0,0,31,0,0,30,31
1,Success,SILAT AVENUE,3,Non-Landed,27539.617540,AVENUE SOUTH RESIDENCE,28778.744350,United Venture Development (Silat) Pte Ltd,RCR,1122,0,0,0,1074,0,0,1074,1074
2,Success,DE SOUZA AVENUE,21,Non-Landed,20969.598000,VERDALE,36396.627440,C&C (JJK) Pte Ltd,RCR,1122,0,0,0,258,0,0,258,258
3,Success,HOUGANG AVENUE 2,19,Non-Landed,33973.310030,THE FLORENCE RESIDENCES,38799.795650,Florence Development Pte Ltd,OCR,1122,0,0,0,1410,0,0,1410,1410
4,Success,HOLLAND ROAD,10,Non-Landed,24512.843400,HYLL ON HOLLAND,32524.393720,FEC Skypark Pte Ltd,CCR,1122,2833,2890,2661,319,8,0,239,319


In [31]:
#Cleaning data

unitSold.fillna(0, inplace = True)

#Dropping irrelevant columns

newunitSold = unitSold.drop(['Result.developerSales.launchedInMonth','Result.street', 'Result.x', 'Result.y', 'Result.marketSegment', 'Result.developerSales.refPeriod', 'Result.developerSales.highestPrice', 'Result.developerSales.lowestPrice', 'Result.developerSales.soldInMonth', 'Result.developerSales.unitsAvail'], axis = 1)
newunitSold

,Status,Result.district,Result.propertyType,Result.project,Result.developer,Result.developerSales.medianPrice,Result.developerSales.launchedToDate,Result.developerSales.soldToDate
0,Success,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,0,31,30
1,Success,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,1074,1074
2,Success,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,1410,1410
3,Success,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,2788,319,251
4,Success,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,3087,455,421
...,...,...,...,...,...,...,...,...
140,Success,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,0,0
141,Success,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,667,667
142,Success,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,2920,558,473
143,Success,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,1498,448,444


In [32]:
#Cleaning data

unitSold1.fillna(0, inplace = True)

#Dropping irrelevant columns

newunitSold1 = unitSold1.drop(['Result.marketSegment','Result.developerSales.launchedInMonth','Result.street', 'Result.x', 'Result.y', 'Result.developerSales.refPeriod', 'Result.developerSales.highestPrice', 'Result.developerSales.lowestPrice', 'Result.developerSales.soldInMonth', 'Result.developerSales.unitsAvail'], axis = 1)
newunitSold1

,Status,Result.district,Result.propertyType,Result.project,Result.developer,Result.developerSales.medianPrice,Result.developerSales.launchedToDate,Result.developerSales.soldToDate
0,Success,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,0,31,30
1,Success,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,1074,1074
2,Success,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,1410,1410
3,Success,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,2675,319,243
4,Success,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,2978,420,410
...,...,...,...,...,...,...,...,...
142,Success,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,0,0
143,Success,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,667,667
144,Success,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,2988,558,468
145,Success,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,0,448,443


In [33]:
#Cleaning data

unitSold2.fillna(0, inplace = True)

#Dropping irrelevant columns

newunitSold2 = unitSold2.drop(['Result.marketSegment','Result.developerSales.launchedInMonth','Result.street', 'Result.x', 'Result.y', 'Result.developerSales.refPeriod', 'Result.developerSales.highestPrice', 'Result.developerSales.lowestPrice','Result.developerSales.soldInMonth', 'Result.developerSales.unitsAvail'], axis = 1)
newunitSold2

,Status,Result.district,Result.propertyType,Result.project,Result.developer,Result.developerSales.medianPrice,Result.developerSales.launchedToDate,Result.developerSales.soldToDate
0,Success,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,0,31,30
1,Success,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,1074,1074
2,Success,21,Non-Landed,VERDALE,C&C (JJK) Pte Ltd,0,258,258
3,Success,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,1410,1410
4,Success,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,2833,319,239
...,...,...,...,...,...,...,...,...
148,Success,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,0,0
149,Success,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,667,667
150,Success,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,2758,558,466
151,Success,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,1441,448,443


In [34]:
#After ensuring that all the above dataframes (unitSold, unitSold1, unitSold2) have same column
#Combine them together into a new dataframe

# or, alternatively, there's the `ignore_index` option in the `pd.concat()` function:
newUnits = pd.concat([newunitSold, newunitSold1, newunitSold2], ignore_index=True)

newUnits = newUnits.reset_index(drop=True)

newUnits

,Status,Result.district,Result.propertyType,Result.project,Result.developer,Result.developerSales.medianPrice,Result.developerSales.launchedToDate,Result.developerSales.soldToDate
0,Success,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,0,31,30
1,Success,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,1074,1074
2,Success,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,1410,1410
3,Success,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,2788,319,251
4,Success,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,3087,455,421
...,...,...,...,...,...,...,...,...
440,Success,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,0,0
441,Success,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,667,667
442,Success,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,2758,558,466
443,Success,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,1441,448,443


In [35]:
#Finding the number of Available units by doing LaunchedToDate - SoldToDate

newUnits['propertyUnitsAvail'] = newUnits['Result.developerSales.launchedToDate'] - newUnits['Result.developerSales.soldToDate']

newUnits

,Status,Result.district,Result.propertyType,Result.project,Result.developer,Result.developerSales.medianPrice,Result.developerSales.launchedToDate,Result.developerSales.soldToDate,propertyUnitsAvail
0,Success,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,0,31,30,1
1,Success,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,1074,1074,0
2,Success,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,1410,1410,0
3,Success,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,2788,319,251,68
4,Success,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,3087,455,421,34
...,...,...,...,...,...,...,...,...,...
440,Success,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,0,0,0
441,Success,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,667,667,0
442,Success,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,2758,558,466,92
443,Success,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,1441,448,443,5


In [36]:
#Dropping irrelevant columns

newUnitsSold = newUnits.drop(['Status','Result.developerSales.launchedToDate','Result.developerSales.soldToDate','Result.developerSales.medianPrice'], axis = 1)

newUnitsSold

,Result.district,Result.propertyType,Result.project,Result.developer,propertyUnitsAvail
0,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,1
1,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0
2,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0
3,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,68
4,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,34
...,...,...,...,...,...
440,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0
441,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0
442,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,92
443,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,5


In [37]:
#Renaming columns

newUnitsSold.rename(columns = {'Result.district':'districtNo'}, inplace = True)
newUnitsSold.rename(columns = {'Result.propertyType':'propertyType'}, inplace = True)
newUnitsSold.rename(columns = {'Result.project':'propertyProjectName'}, inplace = True)
newUnitsSold.rename(columns = {'Result.developer':'propertyDeveloperName'}, inplace = True)

#Inserting new columns

newUnitsSold.insert(5, 'statusBuyRent', 'Buy')
newUnitsSold.insert(6, 'propertyPrivatePublic', 'Private')

newUnitsSold

,districtNo,propertyType,propertyProjectName,propertyDeveloperName,propertyUnitsAvail,statusBuyRent,propertyPrivatePublic
0,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,1,Buy,Private
1,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,Buy,Private
2,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,Buy,Private
3,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,68,Buy,Private
4,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,34,Buy,Private
...,...,...,...,...,...,...,...
440,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,Buy,Private
441,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,Buy,Private
442,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,92,Buy,Private
443,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,5,Buy,Private


# ALL DATASETS (AS PER REQUIREMENTS)

# 1. Private/HDB Property Transactions

In [38]:
newpvtResidential

,propertyPrice,propertyType,districtNo,propertyTenure,propertyPrivatePublic,statusBuyRent
0,4600000,Terrace,5,999.0,Private,Buy
1,2630000,Terrace,5,999.0,Private,Buy
2,1880000,Strata Terrace,5,999.0,Private,Buy
3,5500000,Semi-detached,5,999.0,Private,Buy
4,5000000,Semi-detached,5,999.0,Private,Buy
...,...,...,...,...,...,...
23595,1988000,Condominium,3,89.0,Private,Buy
23596,1500000,Condominium,3,89.0,Private,Buy
23597,1086000,Condominium,3,89.0,Private,Buy
23598,1150000,Condominium,3,89.0,Private,Buy


In [39]:
NewhdbDetails

,propertyTenure,propertyPrice,propertyLevels,propertyType,propertyPrivatePublic,districtNo,statusBuyRent
0,61.33,232000.0,10 TO 12,Improved 2 ROOM,Public,20,Buy
1,60.58,250000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
2,62.42,262000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
3,62.08,265000.0,04 TO 06,New Generation 3 ROOM,Public,20,Buy
4,62.42,265000.0,01 TO 03,New Generation 3 ROOM,Public,20,Buy
...,...,...,...,...,...,...,...
99995,92.50,500000.0,16 TO 18,Model A 4 ROOM,Public,22,Buy
99996,79.08,500000.0,01 TO 03,Improved 5 ROOM,Public,22,Buy
99997,92.58,500000.0,04 TO 06,Model A 4 ROOM,Public,22,Buy
99998,85.92,505000.0,13 TO 15,Premium Apartment 5 ROOM,Public,22,Buy


# 2. Private Non-Landed Residential Property

In [40]:
newnonLanded

,propertyProjectName,rentalRefPeriod,rentalPriceSqft,districtNo,statusBuyRent,propertyPrivatePublic
0,MERAWOODS,2020Q3,1930.0,23,Rent,Private
1,MERAWOODS,2020Q1,1910.0,23,Rent,Private
2,MERAWOODS,2021Q4,2170.0,23,Rent,Private
3,MERAWOODS,2021Q2,2060.0,23,Rent,Private
4,MERAWOODS,2022Q1,2070.0,23,Rent,Private
...,...,...,...,...,...,...
7485,THOMSON V TWO,2021Q4,4430.0,20,Rent,Private
7486,THOMSON V TWO,2021Q1,3910.0,20,Rent,Private
7487,THOMSON V TWO,2022Q3,4970.0,20,Rent,Private
7488,UPTOWN @ FARRER,2022Q2,6540.0,8,Rent,Private


# 3. Private Residential Project in the Pipeline

In [41]:
newprojPipeline

,districtNo,propertyProjectName,propertyDeveloperName,propertyUnitsAvail,statusBuyRent,propertyPrivatePublic
0,23,The Botany At Dairy Farm,Sim Lian JV (Dairy Farm) Pte Ltd,386,Buy,Private
1,10,Boulevard 88,Granmil Holdings Pte Ltd,154,Buy,Private
2,19,Riverfront Residences,Rio Casa Venture Pte Ltd,1472,Buy,Private
3,19,The Florence Residences,Florence Development Pte Ltd,1410,Buy,Private
4,5,Normanton Park,Kingsford Huray Development Pte Ltd,1862,Buy,Private
...,...,...,...,...,...,...
77,1,Skywaters Residences,Perennial Shenton Property Pte Ltd,215,Buy,Private
78,24,Copen Grand,Taurus Properties SG Pte Ltd,639,Buy,Private
79,18,Tenet,TQS Development Pte Ltd,618,Buy,Private
80,26,Lentor Modern,Lentor Central Pte Ltd/Lentor Modern Pte Ltd,605,Buy,Private


# 4 Private Residentials Property (Sold by Developers)

In [42]:
newUnitsSold

,districtNo,propertyType,propertyProjectName,propertyDeveloperName,propertyUnitsAvail,statusBuyRent,propertyPrivatePublic
0,15,Non-Landed,RYMDEN 77,QHS Development Pte Ltd,1,Buy,Private
1,3,Non-Landed,AVENUE SOUTH RESIDENCE,United Venture Development (Silat) Pte Ltd,0,Buy,Private
2,19,Non-Landed,THE FLORENCE RESIDENCES,Florence Development Pte Ltd,0,Buy,Private
3,10,Non-Landed,HYLL ON HOLLAND,FEC Skypark Pte Ltd,68,Buy,Private
4,3,Non-Landed,RIVIERE,Frasers Property Quayside Pte Ltd,34,Buy,Private
...,...,...,...,...,...,...,...
440,14,Non-Landed,GEMS VILLE,East Asia Geylang Development Pte Ltd,0,Buy,Private
441,13,Non-Landed,THE WOODLEIGH RESIDENCES,The Woodleigh Residences Pte Ltd/ The Woodleig...,0,Buy,Private
442,7,Non-Landed,MIDTOWN MODERN,Guoco Midtown II Pte Ltd/Midtown Modern Pte Ltd,92,Buy,Private
443,27,Non-Landed,THE WATERGARDENS AT CANBERRA,United Venture Development (2020) Pte Ltd,5,Buy,Private
